# Image Export Workflow

In [1]:

# Select the kernel observatorio-ipa(Python 3.12.7) .venv/Scripts/python
import json
import ee
import ee.batch
import ee.data
import logging
import importlib
from pathlib import Path
from datetime import date
from gee_toolbox.gee import assets as toolbox_assets
import sqlite3

from observatorio_ipa.services import connections
from observatorio_ipa.core.workflows import wflows_connections
from observatorio_ipa.core import config
from observatorio_ipa.core.defaults import *
from observatorio_ipa.utils import dates as utils_dates
from observatorio_ipa.services.gee import dates as gee_dates, assets as gee_assets
from observatorio_ipa.services.gee.processes import binary, merge
from observatorio_ipa.core.workflows.images import monthly_export



logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [19]:
# general settings
#! JS code shows: users/observatorionieves/Cuencas/Andes # Verified on 2025-06-30

settings = {
    "user": "osn-imageautomation-dev@ee-observatorionieves.iam.gserviceaccount.com",
    "service_credentials_file": Path(
        "../secrets/ee-observatorionieves-288939dbc1cf.json"
    ),
    "monthly_assets_path": Path(
        "projects/ee-observatorionieves/assets/Test/MODIS/Andes_MCDS4S5_Daily"
    ),
    "monthly_image_prefix": "Andes_MCDS4S5_Daily",
    "months_list": [
        "2025-01",
        "2025-02",
        "2025-03",
        "2025-04",
        "2025-05",
        "2025-06",
        "2025-07",
        "2025-08",
    ],
    "aoi_asset_path": Path("projects/ee-observatorionieves/assets/Modules/Andes"),
    "dem_asset_path": Path(
        "projects/ee-observatorionieves/assets/Modules/DEM_SRTM_reproj_MODIS_463_Andes"
    ),
}

In [3]:
# Connect to Google Earth Engine

runtime_service_account = connections.GoogleServiceAccount(
    settings["service_credentials_file"].as_posix(),
)
connections.connect_to_gee(runtime_service_account)



## Verify Access to target IC

In [4]:
gee_assets.check_asset_exists(
        path=settings["monthly_assets_path"].as_posix(), asset_type="IMAGE_COLLECTION"
    )

True

### Test Function

In [20]:
# Get Individual Vars in case we need them later
monthly_collection_path = settings["monthly_assets_path"].as_posix()
name_prefix = settings["monthly_image_prefix"]
aoi_path = settings["aoi_asset_path"].as_posix()
dem_path = settings["dem_asset_path"].as_posix()
months_list = settings["months_list"]

In [ ]:
monthly_export_results = monthly_export.monthly_img_export_proc(
    monthly_collection_path=monthly_collection_path,
    name_prefix=name_prefix,
    aoi_path=aoi_path,
    dem_path=dem_path,
    months_list=months_list,
)

In [12]:
monthly_export_results

{'frequency': 'monthly',
 'images_pending_export': ['2024-11', '2024-12', '2025-01'],
 'images_excluded': [{'2024-12': 'Pending buffer dates - last available image: 2025-01-01 (Aqua)'},
  {'2025-01': 'Current month'}],
 'images_to_export': ['2024-11'],
 'export_tasks': [{'task': 'mock_task',
   'image': 'Mock_prefix_2024_11',
   'target': 'GEE Asset',
   'status': 'mock_created'}]}

## Step by Step (Debug)

In [10]:
from observatorio_ipa.core.workflows.images.monthly_export import _fix_name_prefix, _monthly_images_pending_export, _check_months_are_complete, _ic_monthly_mean
from observatorio_ipa.services.gee.exports import ExportTaskList

In [21]:
logger.info("Starting Monthly Export Process")

    # Fix name prefix
name_prefix = _fix_name_prefix(name_prefix)

results_dict = {
    "frequency": "monthly",
    "initial_export_plan": [],  # aka expected dates ['2003-01', '2003-02', ...]
    "images_pending_export": [],  # initial_plan - already_exported ['2003-01', ...]
    "images_excluded": [],  # Single exclusion reason [{month_: exclusion_str}]
    "images_to_export": [],  # aka final export plan
    "export_tasks": ExportTaskList(),  #
}

# Get Terra and Aqua image collections, AOI and DEM image
ee_terra_ic = ee.imagecollection.ImageCollection(DEFAULT_TERRA_COLLECTION)
ee_aqua_ic = ee.imagecollection.ImageCollection(DEFAULT_AQUA_COLLECTION)
ee_aoi_fc = ee.featurecollection.FeatureCollection(aoi_path)
ee_dem_img = ee.image.Image(dem_path)
trailing_days = 2  # hardcode for now
leading_days = 2  # hardcode for now

# Determine months expected to be exported
if months_list:
    initial_export_plan = months_list
else:
    initial_export_plan = utils_dates.create_ym_seq(
        start_date=date.fromisoformat(DEFAULT_START_DT), end_date=date.today()
    )

# ***********************************
# * EXCLUDE IMAGES ALREADY EXPORTED *
# ***********************************

images_pending_export = _monthly_images_pending_export(
    expected_dates=initial_export_plan,
    monthly_collection_path=monthly_collection_path,
    name_prefix=name_prefix,
)

# images_pending_export = initial_export_plan

#! WARNING: This might print a very long list
logger.info(f"Images pending export: {images_pending_export}")
results_dict["images_pending_export"].extend(images_pending_export)

# if no explicit list of months (month_list) was provided, assume all pending images was the starting point
if months_list:
    excluded_existing = list(set(initial_export_plan) - set(images_pending_export))
    excluded_existing = [
        {_month: "already exported"} for _month in excluded_existing
    ]
    if excluded_existing:
        logger.info(f"Images excluded: {excluded_existing}")
        results_dict["images_excluded"].extend(excluded_existing)
else:
    initial_export_plan = list(
        set(initial_export_plan) - set(images_pending_export)
    )

results_dict["initial_export_plan"].extend(initial_export_plan)

# Terminate early if no images pending of export
# if not images_pending_export:
#     return results_dict

In [22]:
print(results_dict)

{'frequency': 'monthly', 'initial_export_plan': ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08'], 'images_pending_export': ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08'], 'images_excluded': [], 'images_to_export': [], 'export_tasks': ExportList(export_tasks=[])}


In [23]:
# **********************************************
# * EXCLUDE IMAGES NOT AVAILABLE IN TERRA/AQUA *
# **********************************************

# Exclude if:
# - Current month
# - Month not 'Complete' in either Terra/Aqua (Give time to pending source to catch up)

terra_image_dates = gee_dates.get_collection_dates(ee_terra_ic)
aqua_image_dates = gee_dates.get_collection_dates(ee_aqua_ic)

t_availability_results = _check_months_are_complete(
    months=images_pending_export,
    reference_dates=terra_image_dates,
    trailing_days=trailing_days,
    leading_days=leading_days,
)
a_availability_results = _check_months_are_complete(
    months=images_pending_export,
    reference_dates=aqua_image_dates,
    trailing_days=trailing_days,
    leading_days=leading_days,
)

# Months that are 'complete'
t_complete = [
    _month["month"]
    for _month in t_availability_results
    if _month["status"] == "complete"
]
a_complete = [
    _month["month"]
    for _month in a_availability_results
    if _month["status"] == "complete"
]

# Months that are pending completion
t_pending_completion = [
    _month["month"]
    for _month in t_availability_results
    if _month["status"] == "exclude"
    and _month["exclusion_reason"].startswith("Pending")
]

a_pending_completion = [
    _month["month"]
    for _month in a_availability_results
    if _month["status"] == "exclude"
    and _month["exclusion_reason"].startswith("Pending")
]

complete_in_either = set(t_complete + a_complete)
pending_in_either = set(t_pending_completion + a_pending_completion)

# - initial_export_plan: if month_list is provided
# - images_pending_export: after excluding images already exported (same as initial_export_plan if month_list not provided)
# - images_to_export: is the final export plan

# Excluding pending months to give time to either Terra or Aqua to catch up
images_to_export: list[str] = list(complete_in_either - pending_in_either)
excluded_not_available: list[str] = list(
    set(images_pending_export) - set(images_to_export)
)

# find exclusion reason from Terra/Aqua availability, keep only first reason
t_exclusions = [
    {**i, "source": "Terra"}
    for i in t_availability_results
    if i["status"] == "exclude"
]
a_exclusions = [
    {**i, "source": "Aqua"}
    for i in a_availability_results
    if i["status"] == "exclude"
]
all_exclusions = t_exclusions + a_exclusions

single_exclusion_reasons = []
for month_ in excluded_not_available:
    month_exclusions = [item for item in all_exclusions if item["month"] == month_]
    if month_exclusions:
        exclusion_str = f"{month_exclusions[0]['exclusion_reason']} ({month_exclusions[0]['source']})"
        single_exclusion_reasons.append({month_: exclusion_str})

if single_exclusion_reasons:
    results_dict["images_excluded"].extend(single_exclusion_reasons)
    for month_ in single_exclusion_reasons:
        logger.info(f"Image excluded: {month_}")

if images_to_export:
    results_dict["images_to_export"] = images_to_export
    logger.info(f"Images to export: {images_to_export}")
# else:
#     # Terminate early if no images to export|
#     return results_dict

In [24]:
# ***************************************
# * FILTER TERRA/AQUA IMAGE COLLECTIONS *
# ***************************************

# Keep only dates of interest in Terra and Aqua image collections
# Months can be non-sequential so creating sequences for each target month (including buffers) and joining
ic_filter_dates = set()  # creating a Set to avoid duplicates
for month_ in images_to_export:
    month_dates = utils_dates.create_period_seq(
        month_, trailing_days=trailing_days, leading_days=leading_days
    )
    ic_filter_dates.update(month_dates)

ic_filter_dates = list(ic_filter_dates)  # Back to list to sort
ic_filter_dates.sort()

ee_filtered_terra_ic = gee_dates.filter_collection_by_dates(
    ee_terra_ic, ic_filter_dates
)
ee_filtered_aqua_ic = gee_dates.filter_collection_by_dates(
    ee_aqua_ic, ic_filter_dates
)

In [14]:
# Test how many images in each collection
terra_image_dates = gee_dates.get_collection_dates(ee_filtered_terra_ic)
print(f"Filtered Terra images: {len(terra_image_dates)}")
aqua_image_dates = gee_dates.get_collection_dates(ee_filtered_aqua_ic)
print(f"Filtered Aqua images: {len(aqua_image_dates)}")

Filtered Terra images: 216
Filtered Aqua images: 216


In [15]:
print(min(terra_image_dates))
print(min(aqua_image_dates))

2024-12-30
2024-12-30


In [25]:
from observatorio_ipa.services.gee.processes import reclass_and_impute

# **********************************************************
# * APPLY MAIN LANDCOVER RECLASSIFICATION & IMPUTE PROCESS *
# **********************************************************

ee_cloud_snow_ic = reclass_and_impute.tac_reclass_and_impute(
    ee_terra_ic=ee_filtered_terra_ic,
    ee_aqua_ic=ee_filtered_aqua_ic,
    ee_aoi_fc=ee_aoi_fc,
    ee_dem_img=ee_dem_img,
)

### Export Daily images

In [26]:
daily_images = ["2025-01-30", "2025-01-31", "2025-02-01", "2025-02-02", "2025-02-03"]

In [27]:
export_tasks = ExportTaskList()
for day_ in daily_images:
    image_name = name_prefix + day_
    try:
        # Create export task for each daily image
        ee_image = ee_cloud_snow_ic.filterDate(day_).first()
        ee_task = ee.batch.Export.image.toAsset(
            image=ee_image,
            description=image_name,
            assetId=Path(monthly_collection_path, image_name).as_posix(),
            region=ee_aoi_fc.geometry(),
            scale=DEFAULT_SCALE,
            crs=DEFAULT_CHI_PROJECTION,
            maxPixels=180000000,
        )
        # Save to list of Tasks
        export_tasks.add_task(
            type="image",
            name=image_name,
            target="gee",
            path=Path(monthly_collection_path),
            task=ee_task,
            # task_status="mock_created",
        )
        logger.debug(f"Export task created for image: {image_name}")

    except Exception as e:
        # If error, save an empty task with the error for reference
        export_tasks.add_task(
            type="image",
            name=image_name,
            target="gee",
            path=Path(monthly_collection_path),
            task=None,
            task_status="failed_to_create",
            error=str(e),
        )
        logger.debug(f"Export task creation failed for image: {image_name}")
        raise e

In [28]:
print(export_tasks.pretty_summary())

| Status      | Count |
| NOT_STARTED |   5   |


In [29]:
export_tasks.start_exports()

{'PENDING': 5}

In [31]:
export_tasks.query_status()
print(export_tasks.pretty_summary())

| Status  | Count |
| PENDING |   5   |


In [32]:
print(export_tasks)

(type=image, name=Andes_MCDS4S5_Daily_2025-01-30, target=gee, status=PENDING, task_status=RUNNING)
(type=image, name=Andes_MCDS4S5_Daily_2025-01-31, target=gee, status=PENDING, task_status=RUNNING)
(type=image, name=Andes_MCDS4S5_Daily_2025-02-01, target=gee, status=PENDING, task_status=RUNNING)
(type=image, name=Andes_MCDS4S5_Daily_2025-02-02, target=gee, status=PENDING, task_status=READY)
(type=image, name=Andes_MCDS4S5_Daily_2025-02-03, target=gee, status=PENDING, task_status=READY)


#### Manual RUN of Reclass and Impute to verify where the "Out of Memory" issue is comming from


In [16]:
ee_terra_ic = ee_filtered_terra_ic
ee_aqua_ic = ee_filtered_aqua_ic
# ee_aoi_fc = ee_aoi_fc
# ee_dem_img = ee_dem_img

In [17]:
# step 0: reclass snow landcover (Should keep all images in the collection)
ee_terra_reclass_ic = binary.ic_snow_landcover_reclass(ee_terra_ic, ee_aoi_fc, 40)
ee_aqua_reclass_ic = binary.ic_snow_landcover_reclass(ee_aqua_ic, ee_aoi_fc, 40)

In [18]:
# Test how many images in each collection
terra_image_dates = gee_dates.get_collection_dates(ee_terra_reclass_ic)
print(f"Filtered Terra images: {len(terra_image_dates)}")
aqua_image_dates = gee_dates.get_collection_dates(ee_aqua_reclass_ic)
print(f"Filtered Aqua images: {len(aqua_image_dates)}")

Filtered Terra images: 216
Filtered Aqua images: 216


In [19]:
from observatorio_ipa.services.gee.processes.imputation import temporal, spatial_4, spatial_8

# step 1: merge collections (Terra & Aqua)
ee_merged_ic = merge.merge(ee_terra_reclass_ic, ee_aqua_reclass_ic)

# step 2: Impute TAC values from temporal time series
ee_temporal_ic = temporal.ic_impute_tac_temporal(ee_merged_ic)

In [20]:
# Test how many images in each collection
TAC_images = gee_dates.get_collection_dates(ee_temporal_ic)
print(f"Filtered Terra images: {len(TAC_images)}")


Filtered Terra images: 216


In [21]:
# step 3: Impute from spatial neighbors
ee_imputed_ic = spatial_4.ic_impute_TAC_spatial4(ee_temporal_ic)

# step 4: Impute from spatial neighbors and DEM data
ee_imputed_ic = spatial_8.ic_impute_tac_spatial_dem(ee_imputed_ic, ee_dem_img)

In [22]:
# Test how many images in each collection
TAC_images = gee_dates.get_collection_dates(ee_imputed_ic)
print(f"Imputed TAC images: {len(TAC_images)}")

Imputed TAC images: 216


In [23]:
from observatorio_ipa.services.gee.processes.reclass_and_impute import _split_cloud_snow_bands
ee_cloud_snow_ic = ee_imputed_ic.map(_split_cloud_snow_bands).select(
    "Cloud_TAC", "Snow_TAC", "QA_CR"
)

#### Continue Normal Workflow

In [10]:
# Calculate Monthly means for months of interest
ee_monthly_imgs_list = ee.ee_list.List(images_to_export)
ee_monthly_tac_ic = ee.imagecollection.ImageCollection.fromImages(
    ee_monthly_imgs_list.map(
        lambda ee_ym: _ic_monthly_mean(ee_ym, ee_cloud_snow_ic, ee_aoi_fc)
    )
)

In [11]:
monthly_img_dates = images_to_export
monthly_img_dates.sort()
print(f"Months: {monthly_img_dates}")

Months: ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07']


In [ ]:
from observatorio_ipa.services.gee import exports
reload(exports)

In [ ]:
empty_list = exports.ExportTaskList()


In [33]:
export_tasks = ExportTaskList()
for month_ in monthly_img_dates:
    image_name = name_prefix + month_[0:7].replace("-", "_")
    try:
        # Create export task for each monthly image
        ee_image = ee_monthly_tac_ic.filterDate(month_).first()
        ee_task = ee.batch.Export.image.toAsset(
            image=ee_image,
            description=image_name,
            assetId=Path(monthly_collection_path, image_name).as_posix(),
            region=ee_aoi_fc.geometry(),
            scale=DEFAULT_SCALE,
            crs=DEFAULT_CHI_PROJECTION,
            maxPixels=180000000,
        )
        # Save to list of Tasks
        export_tasks.add_task(
            type="image",
            name=image_name,
            target="gee",
            path=Path(monthly_collection_path),
            task=ee_task,
            #task_status="mock_created",
        )
        logger.debug(f"Export task created for image: {image_name}")

    except Exception as e:
        # If error, save an empty task with the error for reference
        export_tasks.add_task(
            type="image",
            name=image_name,
            target="gee",
            path=Path(monthly_collection_path),
            task=None,
            task_status="failed_to_create",
            error=str(e),
        )
        logger.debug(f"Export task creation failed for image: {image_name}")
        raise e

In [34]:
print(export_tasks.pretty_summary())

| Status      | Count |
| NOT_STARTED |   7   |


In [14]:
first_export=export_tasks[0]

In [15]:
print(first_export)
print(first_export.error)

(type=image, name=Andes_MCDS4S5_Yearly_Monthly_2025_01, target=gee, status=NOT_STARTED, task_status=UNSUBMITTED)
None


In [59]:
export_tasks.query_status()
print(export_tasks.pretty_summary())

| Status | Count |
| FAILED |   7   |


In [35]:
logger.info(f"Export tasks created: {len(export_tasks)}")
export_tasks.start_exports()

results_dict["export_tasks"].extend(export_tasks)
# return results_dict

In [36]:
export_list: ExportTaskList = results_dict.get("export_tasks", ExportTaskList())

In [37]:
export_list.query_status()

{'FAILED': 7, 'PENDING': 7}

In [41]:
print(export_list.pretty_summary())

| Status  | Count |
| FAILED  |   7   |
| PENDING |   7   |


In [26]:
first_export = export_list[0]

In [50]:
getattr(first_export.task, 'id', None)

'6F2LPF5CUWMYN2SQYK5FH3LZ'

In [29]:
first_export.task.status()  # Check the status of the task

{'state': 'FAILED',
 'description': 'Andes_MCDS4S5_Yearly_Monthly_2025_01',
 'priority': 100,
 'creation_timestamp_ms': 1755802727907,
 'update_timestamp_ms': 1755802843136,
 'start_timestamp_ms': 1755802733422,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'error_message': 'Export too large: specified 175296036 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.',
 'id': '6F2LPF5CUWMYN2SQYK5FH3LZ',
 'name': 'projects/ee-observatorionieves/operations/6F2LPF5CUWMYN2SQYK5FH3LZ'}

In [42]:
from observatorio_ipa.utils import db

# importlib.reload(db)
db_path = Path("../db/observatorio_ipa.db")
db_path.exists()

True

In [46]:
# db row state should not be the same as status from ExportTask. ExportTask.status is self calculated from task_status.
# whereas the db state refers to the state for polling

DEFAULT_POLLING_INTERVAL_SEC = 15


def _make_db_export_state(export_task: ExportTask) -> str:
    if export_task.status in ["FAILED"]:
        db_state = "FAILED"
    elif export_task.status in ["PENDING", "UNKNOWN"]:
        db_state = "RUNNING"
    else:
        db_state = "COMPLETED"
    return db_state


def add_exportTask_to_db(
    conn: sqlite3.Connection, job_id: str, export_task: ExportTask
):
    now = db.utc_now()
    now_iso = db.datetime_to_iso(now)

    polling_state = _make_db_export_state(export_task)
    conn.execute(
        """INSERT INTO exports (
            id, job_id, state, type, name, target, path, task_id, 
            task_status, next_check_at, poll_interval_sec, 
            created_at, updated_at)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
        (
            export_task.id,
            job_id,
            polling_state,
            export_task.type,
            export_task.name,
            export_task.target,
            export_task.path.as_posix(),
            (export_task.task.id or None),
            export_task.task_status,
            now_iso,
            DEFAULT_POLLING_INTERVAL_SEC,
            now_iso,
            now_iso,
        ),
    )

In [44]:
# Write Job to DB
now = db.utc_now()
job_id = db.new_id()
with db.db(db_path) as conn:
    conn.execute(
        """INSERT INTO jobs (id, job_status, image_export_status, stats_export_status, report_status, created_at, updated_at)
           VALUES (?, ?, ?, ?, ?, ?, ?)""",
        (
            job_id,
            "RUNNING",
            "PENDING",
            "PENDING",
            "PENDING",
            db.datetime_to_iso(now),
            db.datetime_to_iso(now),
        ),
    )

In [47]:
if export_list:
    with db.db(db_path) as conn:
        cur = conn.cursor()
        try:
            for task in export_list:
                add_exportTask_to_db(conn, job_id, task)
            print(f"Inserted {conn.total_changes} export tasks into the database.")
        # Update Job Status in DB

        except Exception as e:
            print(f"Error occurred while inserting export tasks: {e}")

Inserted 14 export tasks into the database.


In [49]:
print(job_id)

d258473f-b7ab-4a50-bc98-dbd91e6853e8


In [14]:
assets_list=toolbox_assets.list_assets("projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly_Monthly")
assets_list = toolbox_assets.get_asset_names(assets_list)

In [15]:
assets_list.sort()
assets_list[-1]

'projects/ee-observatorionieves/assets/MODIS/Andes_MCDS4S5_Yearly_Monthly/Andes_MCDS4S5_Yearly_Monthly_2024_12'

In [16]:
constant_img = ee.image.Image(100).bandNames().getInfo()
print(constant_img)

['constant']


In [34]:
# if you map over a list of dates you need to recast the dates inside the function
# After recasting to date inside the function, you can use this date to filter the image collection
# IF you map over a list of dates and get no images back, the ee.ImageCollection creation will error out
# iterating over a list of dates that don't produce any images, creates a list with a None object. It's size 1 and not an empty list



ee_terra_ic = ee.imagecollection.ImageCollection("MODIS/061/MOD10A1")

keep_dates=[ "2025-03-03"]
# Convert list of dates to ee_list in milliseconds
ee_keep_dates_list = ee.ee_list.List(
        [ee.ee_date.Date(_date) for _date in keep_dates]
    )

# Convert the list of ee.Dates to simple list of numbers. Otherwise, map wont work
keep_dates_list = ee_keep_dates_list.getInfo()
print(f"list of dates: {keep_dates_list}")

if keep_dates_list:
    keep_dates_list = [item["value"] for item in keep_dates_list]
else:
    keep_dates_list = []
ee_keep_dates_list2 = ee.ee_list.List(keep_dates_list)

keep_dates_str_list = ee_keep_dates_list2.getInfo()
print(f"list of ints?: {keep_dates_str_list}")

def get_image(date):
    ee_date = ee.ee_date.Date(date)
    image = ee_terra_ic.filterDate(ee_date).first()
    return image

ee_something = ee_keep_dates_list.map(get_image)
print(type(ee_something))
print(f"num images: {ee_something.size().getInfo()}")
print(f"List Content: {ee_something.getInfo()}")
#ee_something = ee.imagecollection.ImageCollection(ee_keep_dates_list.map(get_image))
# if ee_something:
#     print(f"num images: {ee_something.size().getInfo()}")

list of dates: [{'type': 'Date', 'value': 1740960000000}]
list of ints?: [1740960000000]
<class 'ee.ee_list.List'>
num images: 1
List Content: [None]


In [ ]:
# If after filtering an IC the IC is empty, .first() produces an object of NoneType
ee_image = ee_terra_ic.filterDate("2025-05-01").first()
type(ee_image)
type(ee_image.getInfo())

NoneType

In [37]:
# Function arguments like strings are automatically casted to ee.String objects

keep_dates = ["2025-03-03", "2025-03-04"]
ee_keep_dates_list = ee.ee_list.List([ee.ee_date.Date(_date) for _date in keep_dates])

def func_2_rgs(date, rand_string):
    ee_date = ee.ee_date.Date(date)
    return rand_string

results = ee_keep_dates_list.map(lambda date: func_2_rgs(date, "random_string"))
results.getInfo()

['random_string', 'random_string']

In [58]:
runtime_settings = config.Settings(
    user="osn-imageautomation-dev@ee-observatorionieves.iam.gserviceaccount.com",
    service_credentials_file=Path("../secrets/ee-observatorionieves-288939dbc1cf.json"),
    daily_assets_path=None,
    monthly_assets_path=Path(
        "users/observatorionieves/MODIS/Andes_MCDS4S5_Yearly_Monthly"
    ),
    monthly_image_prefix="Andes_MCDS4S5_Yearly_Monthly",
    months_list=['2025-05', '2025-06'],
    yearly_assets_path=None,
    aoi_asset_path=Path(
        "projects/ee-observatorionieves/assets/Modules/Andes"
    ),  #! JS code shows: users/observatorionieves/Cuencas/Andes # Verified on 2025-06-30
    dem_asset_path=Path(
        "projects/ee-observatorionieves/assets/Modules/DEM_SRTM_reproj_MODIS_463_Andes"
    ),  #! JS code shows: users/observatorionieves//DEM_SRTM_reproj_MODIS_463_D_Andes # Verified on 2025-06-30
)